# Test Tree Command After Fetch

Simple test to verify tree command works after fetch_file

## Test Tree Command

Testing tree command after fetch_file

Setting up BAML

### BAML Setup

Don't worry too much about this setup code - it will make sense later! For now, just know that:
- BAML is a tool for working with language models
- We need some special setup code to make it work nicely in Google Colab
- The `get_baml_client()` function will be used to interact with AI models

In [ ]:
!pip install baml-py==0.202.0 pydantic

In [ ]:
import subprocess
import os

# Try to import Google Colab userdata, but don't fail if not in Colab
try:
    from google.colab import userdata
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

def baml_generate():
    try:
        result = subprocess.run(
            ["baml-cli", "generate"],
            check=True,
            capture_output=True,
            text=True
        )
        if result.stdout:
            print("[baml-cli generate]\n", result.stdout)
        if result.stderr:
            print("[baml-cli generate]\n", result.stderr)
    except subprocess.CalledProcessError as e:
        msg = (
            f"`baml-cli generate` failed with exit code {e.returncode}\n"
            f"--- STDOUT ---\n{e.stdout}\n"
            f"--- STDERR ---\n{e.stderr}"
        )
        raise RuntimeError(msg) from None

def get_baml_client():
    """
    a bunch of fun jank to work around the google colab import cache
    """
    # Set API key from Colab secrets or environment
    if IN_COLAB:
        os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')
    elif 'OPENAI_API_KEY' not in os.environ:
        print("Warning: OPENAI_API_KEY not set. Please set it in your environment.")
    
    baml_generate()
    
    # Force delete all baml_client modules from sys.modules
    import sys
    modules_to_delete = [key for key in sys.modules.keys() if key.startswith('baml_client')]
    for module in modules_to_delete:
        del sys.modules[module]
    
    # Now import fresh
    import baml_client
    return baml_client.sync_client.b


In [ ]:
!baml-cli init

In [ ]:
# Helper function to capture BAML logs in notebook output
import os
from IPython.utils.capture import capture_output

def run_with_baml_logs(func, *args, **kwargs):
    """Run a function and capture BAML logs in the notebook output."""
    # Capture both stdout and stderr
    with capture_output() as captured:
        result = func(*args, **kwargs)
    
    # Display the captured output
    if captured.stdout:
        print(captured.stdout)
    if captured.stderr:
        # BAML logs go to stderr - format them nicely
        print("\n=== BAML Logs ===")
        print(captured.stderr)
        print("=================\n")
    
    return result

# Set BAML log level (options: error, warn, info, debug, trace)
os.environ['BAML_LOG'] = 'info'


Fetching a BAML file

In [ ]:
!curl -fsSL -o baml_src/agent.baml https://raw.githubusercontent.com/humanlayer/12-factor-agents/refs/heads/main/workshops/2025-07-16/./walkthrough/01-agent.baml && cat baml_src/agent.baml

In [ ]:
!tree -I baml_client

The tree command above should show our file structure

Let's fetch another file

In [ ]:
!curl -fsSL -o baml_src/tool_calculator.baml https://raw.githubusercontent.com/humanlayer/12-factor-agents/refs/heads/main/workshops/2025-07-16/./walkthrough/02-tool_calculator.baml && cat baml_src/tool_calculator.baml

In [ ]:
!tree -I baml_client

Now we should see both BAML files in the tree